In [48]:
from aide_design.play import*
import math

import os
print(os.getcwd())


def import_file(full_path_to_module):
    try:
        import os
        module_dir, module_file = os.path.split(full_path_to_module)
        module_name, module_ext = os.path.splitext(module_file)
        save_cwd = os.getcwd()
        os.chdir(module_dir)
        module_obj = __import__(module_name)
        module_obj.__file__ = full_path_to_module
        globals()[module_name] = module_obj
        os.chdir(save_cwd)
    except:
        raise ImportError

import_file('/Users/cynthia/Documents/GitHub/UASB/UASB_size.py')

/Users/cynthia/Documents/GitHub/UASB


ImportError: 

# Tube Settler Calculations
## Determine the size of a tube settler in a UASB reactor

## Critical Assumptions

- assume capture velocity is 1.2 mm/s
- assume diameter is 4 inches

In [31]:
diam = 3 * u.ft

area_reactor = np.pi * diam**2 / 4

v_up = (0.03617 * u.L/u.s) / area_reactor

print(v_up. to (u.mm/u.s))


diam_tube = 4 * u. inch


# upflow velocity through the tube settler
v = 0.03617 * u.L/u.s / pc.area_circle(diam_tube)

print (v.to(u.mm/u.s))

0.05508 millimeter / second
4.461 millimeter / second


In [28]:
V_reactor_up = 1 * u.mm/u.s    #we have this
V_capture = 1.2 * u.mm/u.s       #needs research?!
angle = 60 * u.deg




# slide Q/As
def len_tube(diam, angle, v_up, v_cap):
    len = ((v_up * np.pi * diam) / (4 * v_cap * math.cos(angle)))
    return len.to(u.inch)

print(len_tube(diam_tube, angle, v_up, V_capture))

0.2884 inch


tube selttler into the reactor a smidge

length
    - capture velocity (0.05 mm/s or set to upflow veloctiy through the tube settler)
    - **water elevation control, needs to be long enough to set height ontop of water lid** this sets the length
    - multiple exits? potentially not necessary 
diameter
    - S must be 2.5 cm  
height 

dont know v capture and v up but we can get a ratio of the two 